In [1]:
import json
import os
import subprocess
import time

from typing import List

from benchmark_utils import *

import sys
root_dir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# Evaluation Vector
sys.path.append(f'{root_dir}/line_detection_evaluation/vector/linelet_modified')
from line_detection_evaluation.vector.linelet_modified.evaluate_line_segment import evaluate_line_segment_complete
from line_detection_evaluation.vector.linelet_modified.utils import eval_param_struct
# Evaluation Pixel
from line_detection_evaluation.pixel.icdar_2013.evaluate_staff_line import icdar_2013_evaluate_pixel
from line_detection_evaluation.pixel.icdar_2011.evaluate_staff_line import icdar_2011_evaluate_pixel
from line_detection_evaluation.pixel.coco.eval_coco import eval_coco_path, eval_coco_path_score

In [2]:
def load_lsd_score():
    with open('lsd_score.json') as json_file:
        data = json.load(json_file)
    return data


def dump_lsd_score(lsd_score):
    with open('lsd_score.json', 'w') as outfile:
        json.dump(lsd_score, outfile, indent=4)


In [3]:
def lem(gt: str, cd: str) -> dict:
    try:
        evaluation_parameters = eval_param_struct(
            30, 3.14 * 5 / 180, .75, False, 300)
        (p, r, iou, fsc) = evaluate_line_segment_complete(
            gt, cd, evaluation_parameters)
    except:
        (p, r, iou, fsc) = (None, None, None, None)

    return {
        "precision": p,
        "recall": r,
        "iou": iou,
        "fscore": fsc
    }


def lems(gt: str, cd: str) -> dict:
    try:
        evaluation_parameters = eval_param_struct(
            30, 3.14 * 5 / 180, .75, True, 300)
        (p, r, iou, fsc) = evaluate_line_segment_complete(
            gt, cd, evaluation_parameters)
    except:
        (p, r, iou, fsc) = (None, None, None, None)

    return {
        "precision": p,
        "recall": r,
        "iou": iou,
        "fscore": fsc
    }


def coco(gt: str, cd: str) -> dict:
    try:
        (PQ, SQ, RQ) = eval_coco_path_score(gt, cd)
    except:
        (PQ, SQ, RQ) = (None, None, None)
    return {
        "PQ": PQ,
        "SQ": SQ,
        "RQ": RQ
    }


def icdar2013(gt: str, cd: str) -> dict:
    try:
        p, r, fscore = icdar_2013_evaluate_pixel(gt, cd)
    except:
        p, r, fscore = (None, None, None)
    return {
        "precision": p,
        "recall": r,
        "fscore": fscore
    }


evaluations = {
    ".csv": {
        "lem": lem,
        "lems": lems,
    },
    ".png": {
        "coco": coco,
        "icdar2013": icdar2013,
    }
}


In [4]:
def update_partial(method : List[str], experience : str, type : str, build : bool=False):
    """
    Perform the evaluation of the method on the dataset

    :param method: The method to evaluate
    :param experience: The experience to evaluate
    :param type: The type of the experience to evaluate
    :param build: If false, the evaluation is performed, otherwise only the json is created
    :return: The evaluation of the method on the dataset
    """
    ret = {}

    path = os.path.join(dataset_folder, experience, type, 'input')
    gt_path = path.replace('input', 'ground truth')
    out_path = path.replace('input', 'output')

    try:
        files = [os.path.splitext(f)[0] for f in os.listdir(path)]
        for file in files:
            ret[file] = {}
            for eval_ext, evalutions_ext in evaluations.items():
                for eval_name, eval in evalutions_ext.items():
                    if build:
                        gt_file = None
                        cd_file = None
                    else:
                        gt_file = os.path.join(gt_path, file + eval_ext)
                        cd_file = os.path.join(
                            out_path, method, file + eval_ext)
                    ret[file][eval_name] = eval(gt_file, cd_file)
    except:
        print("Error in " + method + " " + experience + " " + type)
        return {}
    return ret


In [5]:
def build_json_result() -> dict:
    """
    Build the json result of the evaluation

    :return: The json result of the evaluation
    """
    json_out = {}

    for method in methods:
        json_out[method] = {}
        for experience in experiences:
            json_out[method][experience] = {}
            for type in ["train", "test"]:
                json_out[method][experience][type] = update_partial(
                    method, experience, type, evaluations, True)
    return json_out

# json_out = build_json_result()

# with open(json_out_filename, 'w') as f:
#     json.dump(json_out, f, indent=2)


In [6]:
def update_experience_time(experience : str, methods : List[str], types:List[str]=["train", "test"]) -> None:
    """
    Update the time information in the json file

    :param experience: The experience to update
    :param methods: The methods to update
    :param type: The type of the experience to update
    :return: None
    """
    ret = load_lsd_score()

    for type in types:
        time_json = os.path.join(dataset_folder, experience, type, 'time.json')
        if not os.path.exists(time_json):
            print(f"Time file not found : {time_json}")
            continue
        
        with open(time_json) as json_file:
            time_data = json.load(json_file)

        for method in methods:
            if not method in time_data:
                continue

            for file in ret[method][experience][type]:
                ret[method][experience][type][file]["time"] = {}
                try :
                    ret[method][experience][type][file]["time"]["sec"] = time_data[method][file]
                except:
                    ret[method][experience][type][file]["time"]["sec"] = None

    dump_lsd_score(ret)

def update_experience_evaluation(experience:str, methods:List[str], types:List[str]=['train', 'test']) -> None:
    """
    Update the evaluation of the methods on the dataset

    :param experience: The experience to update
    :param methods: The methods to update
    :param types: The types of the experience to update
    :return: None
    """
    ret = load_lsd_score()
    t0 = time.time()
    for method in methods:
        print(method, time.time() - t0)
        for type in types:
            ret[method][experience][type] = update_partial(
                method, experience, type)
    dump_lsd_score(ret)

def update_experience(experience:str, methods:List[str], types:List[str]) -> None:
    """
    Update the evaluation and the time duration of the methods on the dataset in the json file

    :param experience: The experience to update
    :param methods: The methods to update
    :param types: The types of the experience to update
    :return: None
    """
    update_experience_time(experience, methods, types)
    update_experience_evaluation(experience, methods, types)

# Update experiences

In [7]:
def disp(list: List[str], name: str) -> None:
    print(f"{name}: ", end="")
    for item in list:
        print(item, end=" ")
    print()


disp(experiences, "experiences")
disp(sta_labels, "sta_labels")
disp(predictor_labels, "predictor_labels")

experiences: maps music_sheets trade_directories 
sta_labels: edlines ocv_hough cannylines lsd lsd_m elsed ag3line 
predictor_labels: Last observation SMA EMA Double exponential Kalman One euro 


In [8]:
# update_experience('trade_directories', sta_labels, ['train', 'test']) # 1 hours 10 minutes
update_experience('trade_directories', predictor_labels, ['train']) # 1 hours 10 minutes
# update_experience('music_sheets',predictor_labels, ['train', 'test']) # 8 hours

# update_experience_time('maps',predictor_labels, ['train']) # Only qualitative evaluation

Last observation 2.86102294921875e-06
SMA 0.3782026767730713
EMA 0.7154903411865234
Double exponential 1.0589468479156494
Kalman 1.5555832386016846
One euro 1.9646663665771484
